In [1]:
# stardist / tensorflow env variables setup
import os
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from pathlib import Path

import numpy as np
from tqdm import tqdm
from rich.pretty import pprint


from ultrack import track, to_tracks_layer, tracks_to_zarr
from ultrack.imgproc import normalize
from ultrack.utils import estimate_parameters_from_labels, labels_to_contours
from ultrack.utils.array import array_apply
from ultrack.config import MainConfig

In [2]:
from tracktour import load_tiff_frames

im_path = '/home/ddon0001/PhD/data/cell_tracking_challenge/ST/Fluo-N2DL-HeLa/01'
label_path = '/home/ddon0001/PhD/data/cell_tracking_challenge/ST/Fluo-N2DL-HeLa/01_ST/SEG'

ims = load_tiff_frames(im_path)
labels = load_tiff_frames(label_path)

Loading TIFFs: 100%|██████████| 91/91 [00:00<00:00, 362.11it/s]


In [3]:
detections, edges = labels_to_contours(labels)

Converting labels to contours: 100%|██████████| 92/92 [00:02<00:00, 45.43it/s]


In [4]:
config = MainConfig()

In [5]:
track(detection=detections, edges=edges, config=config, overwrite=True)

/tmp/ipykernel_17461/3506774268.py:1: DeprecationWarning: Argument detection is deprecated, please use foreground instead.
  track(detection=detections, edges=edges, config=config, overwrite=True)
Linking nodes.: 100%|██████████| 91/91 [00:01<00:00, 67.42it/s]


Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-14
Using Gurobi solver
Solving ILP batch 0
Constructing ILP ...
Set parameter TimeLimit to value 36000
Solving ILP ...
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Set parameter MIPGap to value 0.001
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 28840 rows, 47562 columns and 88488 nonzeros
Model fingerprint: 0xd0f727ab
Variable types: 0 continuous, 47562 integer (47562 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-12, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 28838 rows and 47558 columns
Presolve time: 0.24s
Presolved: 2 rows, 4 columns

In [6]:
from ultrack.core.export import to_ctc
from pathlib import Path

to_ctc(
    output_dir = Path('/home/ddon0001/PhD/experiments/ultrack/out-of-the-box'),
    config=config,
    overwrite=True
)

Exporting segmentation masks: 100%|██████████| 92/92 [00:00<00:00, 116.82it/s]


In [14]:
from traccuracy import run_metrics
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher
from traccuracy.metrics import CTCMetrics
import pprint
pp = pprint.PrettyPrinter(indent=4)


gt_data = load_ctc_data('/home/ddon0001/PhD/data/cell_tracking_challenge/ST/Fluo-N2DL-HeLa/01_GT/TRA/')
pred_data = load_ctc_data('/home/ddon0001/PhD/experiments/ultrack/')

ctc_results = run_metrics(
    gt_data=gt_data,
    pred_data=pred_data,
    matcher=CTCMatcher(),
    metrics=[CTCMetrics()]
)
pp.pprint(ctc_results)


Evaluating FN edges: 100%|██████████| 8562/8562 [00:00<00:00, 795508.29it/s]


[   {   'matcher': {'name': 'CTCMatcher'},
        'metric': {   'e_weights': {'fn': 1.5, 'fp': 1, 'ws': 1},
                      'name': 'CTCMetrics',
                      'v_weights': {'fn': 10, 'fp': 1, 'ns': 5}},
        'results': {   'AOGM': 1273.5,
                       'DET': 0.9905660377358491,
                       'TRA': 0.9871665675732871,
                       'fn_edges': 255,
                       'fn_nodes': 74,
                       'fp_edges': 9,
                       'fp_nodes': 0,
                       'ns_nodes': 15,
                       'ws_edges': 67},
        'version': '0.1.1.dev67+g4160403'}]


In [17]:
# we now configure the tracking parameters as per the example in https://github.com/royerlab/ultrack/blob/main/examples/stardist_2d/2d_tracking.ipynb
# numbers copied directly - this is kinda broken though because our labels aren't like the stardist labels

config.segmentation_config.min_area = 50
config.segmentation_config.max_area = 950
config.segmentation_config.n_workers = 8

config.linking_config.max_distance = 25
config.linking_config.n_workers = 8

config.tracking_config.appear_weight = -1
config.tracking_config.disappear_weight = -1
config.tracking_config.division_weight = -0.1
config.tracking_config.power = 4
config.tracking_config.bias = -0.001
config.tracking_config.solution_gap = 0.0

In [20]:
# using sigma values from the example
new_detections, new_edges = labels_to_contours(labels, sigma=4.0)

# and re-track, and re-save
track(
    detection=new_detections,
    edges=new_edges,
    config=config,
    overwrite=True,
)

to_ctc(
    output_dir = Path('/home/ddon0001/PhD/experiments/ultrack/example-config'),
    config=config,
    overwrite=True
)

Converting labels to contours: 100%|██████████| 92/92 [00:03<00:00, 29.15it/s]
/tmp/ipykernel_17461/4211839314.py:3: DeprecationWarning: Argument detection is deprecated, please use foreground instead.
  track(
Linking nodes.: 100%|██████████| 91/91 [00:05<00:00, 16.95it/s]


Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-14
Using Gurobi solver
Solving ILP batch 0
Constructing ILP ...
Set parameter TimeLimit to value 36000
Solving ILP ...
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Set parameter MIPGap to value 0
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 29913 rows, 51925 columns and 96721 nonzeros
Model fingerprint: 0xa47d1986
Variable types: 0 continuous, 51925 integer (51925 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-06, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 29911 rows and 51921 columns
Presolve time: 0.66s
Presolved: 2 rows, 4 columns, 6 

Exporting segmentation masks: 100%|██████████| 92/92 [00:00<00:00, 122.21it/s]


In [21]:
new_pred_data = load_ctc_data('/home/ddon0001/PhD/experiments/ultrack/example-config')

ctc_results = run_metrics(
    gt_data=gt_data,
    pred_data=new_pred_data,
    matcher=CTCMatcher(),
    metrics=[CTCMetrics()]
)
pp.pprint(ctc_results)

Evaluating FN edges: 100%|██████████| 8562/8562 [00:00<00:00, 691938.94it/s]

[   {   'matcher': {'name': 'CTCMatcher'},
        'metric': {   'e_weights': {'fn': 1.5, 'fp': 1, 'ws': 1},
                      'name': 'CTCMetrics',
                      'v_weights': {'fn': 10, 'fp': 1, 'ns': 5}},
        'results': {   'AOGM': 2564.0,
                       'DET': 0.9756800555619863,
                       'TRA': 0.9741618211683614,
                       'fn_edges': 252,
                       'fn_nodes': 197,
                       'fp_edges': 31,
                       'fp_nodes': 116,
                       'ns_nodes': 3,
                       'ws_edges': 54},
        'version': '0.1.1.dev67+g4160403'}]


In [ ]:
# it got much worse, ok let's try with the stardist labels


# reproduced from https://github.com/royerlab/ultrack/blob/main/examples/stardist_2d/2d_tracking.ipynb

model = StarDist2D.from_pretrained("2D_versatile_fluo")
stardist_labels = np.zeros_like(image, dtype=np.int32)

def predict(frame: np.ndarray, model: StarDist2D) -> np.ndarray:
    """Normalizes and computes stardist prediction."""
    frame = normalize(frame, gamma=1.0)
    labels, _ = model.predict_instances_big(
        frame, "YX", block_size=560, min_overlap=96, show_progress=False,
    )
    return labels

array_apply(
    image,
    out_array=stardist_labels,
    func=predict,
    model=model,
)